In [ ]:
# Import necessary libraries
from flask import Flask, render_template, request
import pickle
import numpy as np
import xgboost as xgb

# Flask app definition
app = Flask(__name__)

# Load the pre-trained XGBoost model
model_filename = r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\xgboost_regression_model_tuned.pkl"
with open(model_filename, 'rb') as model_file:
    model = pickle.load(model_file)

@app.route("/", methods=["GET", "POST"])
def home():
    predicted_value = None

    if request.method == "POST":
         # Get input values from the form
        latitude = float(request.form["latitude"])
             
        population_density = float(request.form["population_density"])

        

        # Prepare the data for prediction
        input_data = np.array([[latitude, population_density]])

        # Get the predicted log-transformed CO2 emission
        predicted_value = model.predict(input_data)[0]



    return render_template("home.html", predicted_value=predicted_value)

# Start Flask app
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)  # `use_reloader=False` to avoid Jupyter Notebook issues


In [ ]:
# Import necessary libraries
from flask import Flask, render_template, request
import pickle
import numpy as np
import xgboost as xgb
import pandas as pd

# Flask app definition
app = Flask(__name__)

# Load the pre-trained XGBoost model
model_filename = r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\xgboost_regression_model_tuned.pkl"
with open(model_filename, 'rb') as model_file:
    model = pickle.load(model_file)
# Load the CSV data with country and latitude information
data_file = r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\data\processed\countrylat.csv"  # Ensure the CSV file is available
df = pd.read_csv(data_file)


@app.route("/", methods=["GET", "POST"])
def home():
    predicted_value = None

    if request.method == "POST":
        # Get input values from the form
        selected_country = request.form["Country"]
        
        # Ensure that latitude_values is properly defined
        latitude_values = df[df['Country'].str.lower() == selected_country.lower()]['Latitude'].values
        
        # Check if latitude was found
        if latitude_values.size > 0:  # Use .size to check if there's at least one value
            latitude = latitude_values[0]  # Extract the single latitude value
            population_density = float(request.form["population_density"])
        
            # Prepare the data for prediction
            input_data = np.array([[latitude, population_density]])
        
            # Get the predicted log-transformed CO2 emission
            predicted_value = model.predict(input_data)[0]
        else:
            latitude = None  # Handle case where the country is not found
            predicted_value = "Country not found or invalid data."



    return render_template("home1.html", predicted_value=predicted_value)

# Start Flask app
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)  # `use_reloader=False` to avoid Jupyter Notebook issues


In [ ]:
from flask import Flask, render_template, request
import pickle
import pandas as pd
import numpy as np

# Flask app definition
app = Flask(__name__)

# Load the pre-trained models and encoder
file_path1 = r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\population_model.pkl"
file_path2 = r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\density_model.pkl"
file_path3 = r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\label_encoder.pkl"

pop_model = pickle.load(open(file_path1, "rb"))
density_model = pickle.load(open(file_path2, "rb"))
le = pickle.load(open(file_path3, "rb"))

# Load the dataset to get country codes for label encoding
df = pd.read_csv(r"C:/Users/Dell/gitproject/CO2-emissions-ML-project/data/processed/merged_full_featured_dataset.csv")
df = df[["Year", "Country"]]

@app.route("/", methods=["GET", "POST"])
def home():
    population_pred = None
    density_pred = None

    if request.method == "POST":
        # Get the input values from the form
        selected_country = request.form["Country"]
        selected_year = int(request.form["Year"])

        # Encode the selected country to match the model input
        country_code = le.transform([selected_country])[0] if selected_country in le.classes_ else None

        if country_code is not None:
            # Prepare the data for prediction
            input_data = np.array([[selected_year, country_code]])

            # Predict population and population density
            population_pred = pop_model.predict(input_data)[0]
            density_pred = density_model.predict(input_data)[0]

        else:
            population_pred = "Invalid country"
            density_pred = "Invalid country"

    return render_template("home2.html", population_pred=population_pred, density_pred=density_pred)

if __name__ == "__main__":
    app.run(debug=True ,  use_reloader=False, port=5001)


In [ ]:
from flask import Flask, render_template, request
import pickle
import pandas as pd
import numpy as np
import xgboost as xgb

# Flask app definition
app = Flask(__name__)

# Load the pre-trained models and encoder
pop_model_path = r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\population_model.pkl"
density_model_path = r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\density_model.pkl"
encoder_path = r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\label_encoder.pkl"
xgb_model_path = r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\xgboost_regression_model_tuned.pkl"

pop_model = pickle.load(open(pop_model_path, "rb"))
density_model = pickle.load(open(density_model_path, "rb"))
le = pickle.load(open(encoder_path, "rb"))
xgb_model = pickle.load(open(xgb_model_path, "rb"))

# Load datasets
df_countries = pd.read_csv(r"C:/Users/Dell/gitproject/CO2-emissions-ML-project/data/processed/merged_full_featured_dataset.csv")[["Year", "Country"]]
df_latitudes = pd.read_csv(r"C:/Users/Dell/gitproject/CO2-emissions-ML-project/data/processed/countrylat.csv")

@app.route("/", methods=["GET", "POST"])
def home():
    population_pred = None
    density_pred = None
    co2_pred = None

    if request.method == "POST":
        selected_country = request.form["Country"]
        selected_year = int(request.form["Year"])

        # Encode country for prediction
        country_code = le.transform([selected_country])[0] if selected_country in le.classes_ else None
        
        if country_code is not None:
            input_data = np.array([[selected_year, country_code]])
            population_pred = pop_model.predict(input_data)[0]
            density_pred = density_model.predict(input_data)[0]
        else:
            population_pred = "Invalid country"
            density_pred = "Invalid country"
        
        # Predict CO2 emissions if population density is available
        latitude_values = df_latitudes[df_latitudes['Country'].str.lower() == selected_country.lower()]['Latitude'].values
        if latitude_values.size > 0:
            latitude = latitude_values[0]
            input_co2_data = np.array([[latitude, density_pred]])
            co2_pred = xgb_model.predict(input_co2_data)[0]
        else:
            co2_pred = "Country not found or invalid data."
    
    return render_template("home4.html", population_pred=population_pred, density_pred=density_pred, co2_pred=co2_pred)

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, port=5001)


In [ ]:
from flask import Flask, render_template, request
import pickle
import pandas as pd
import numpy as np
import xgboost as xgb
import math

# Flask app definition
app = Flask(__name__)

# Load pre-trained models and encoder
pop_model = pickle.load(open(r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\population_model.pkl", "rb"))
density_model = pickle.load(open(r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\density_model.pkl", "rb"))
le = pickle.load(open(r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\label_encoder.pkl", "rb"))
co2_model = pickle.load(open(r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\xgboost_regression_model_tuned.pkl", "rb"))

# Load datasets
df_countries = pd.read_csv(r"C:/Users/Dell/gitproject/CO2-emissions-ML-project/data/processed/merged_full_featured_dataset.csv")
df_latitude = pd.read_csv(r"C:/Users/Dell/gitproject/CO2-emissions-ML-project/data/processed/countrylat.csv")

df_countries = df_countries[["Year", "Country"]]

def predict_population_density(selected_country, selected_year):
    country_code = le.transform([selected_country])[0] if selected_country in le.classes_ else None
    
    if country_code is not None:
        input_data = np.array([[selected_year, country_code]])
        population_pred = pop_model.predict(input_data)[0]
        density_pred = density_model.predict(input_data)[0]
        return population_pred, density_pred
    else:
        return None, None

def predict_co2(selected_country, population_density):
    latitude_values = df_latitude[df_latitude['Country'].str.lower() == selected_country.lower()]['Latitude'].values
    
    if latitude_values.size > 0:
        latitude = latitude_values[0]
        input_data = np.array([[latitude, population_density]])
        co2_pred_log = co2_model.predict(input_data)[0]
        co2_pred = math.exp(co2_pred_log)
        return co2_pred
    return None

@app.route("/", methods=["GET", "POST"])
def home():
    population_pred = None
    density_pred = None
    co2_pred = None

    if request.method == "POST":
        selected_country = request.form.get("Country", "").strip()
        selected_year = request.form.get("Year")
        
        if selected_year:  # First form submission
            selected_year = int(selected_year)
            population_pred, density_pred = predict_population_density(selected_country, selected_year)
        
        if request.form.get("population_density"):  # Second form submission
            population_density = float(request.form["population_density"])
            co2_pred = predict_co2(selected_country, population_density)
    
    return render_template("home4.html", 
                           population_pred=population_pred, 
                           density_pred=density_pred, 
                           co2_pred=co2_pred)

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, port=5001)


In [ ]:
from flask import Flask, render_template, request
import pickle
import pandas as pd
import numpy as np
import xgboost as xgb
import math

# Flask app definition
app = Flask(__name__)

# Load the pre-trained models and encoder for population and density predictions
pop_model = pickle.load(open(r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\population_model.pkl", "rb"))
density_model = pickle.load(open(r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\density_model.pkl", "rb"))
le = pickle.load(open(r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\label_encoder.pkl", "rb"))

# Load the dataset to get country codes for label encoding
df_country_year = pd.read_csv(r"C:/Users/Dell/gitproject/CO2-emissions-ML-project/data/processed/merged_full_featured_dataset.csv")
df_country_year = df_country_year[["Year", "Country"]]

# Load the pre-trained XGBoost model for CO2 emissions prediction
with open(r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\predictive model\xgboost_regression_model_tuned.pkl", 'rb') as model_file:
    co2_model = pickle.load(model_file)

# Load the CSV data with country and latitude information
df_latitude = pd.read_csv(r"C:\Users\Dell\gitproject\CO2-emissions-ML-project\data\processed\countrylat.csv")

@app.route("/", methods=["GET", "POST"])
def home():
    population_pred = None
    density_pred = None
    co2_pred = None

    if request.method == "POST":
        # Get the input values from the form
        selected_country = request.form["Country"]
        selected_year = int(request.form["Year"])

        # Encode the selected country to match the model input for population and density predictions
        country_code = le.transform([selected_country])[0] if selected_country in le.classes_ else None

        if country_code is not None:
            # Prepare the data for population and density predictions
            input_data = np.array([[selected_year, country_code]])

            # Predict population and population density
            population_pred = pop_model.predict(input_data)[0]
            density_pred = density_model.predict(input_data)[0]

            # Use the predicted density for CO2 emissions prediction
            latitude_values = df_latitude[df_latitude['Country'].str.lower() == selected_country.lower()]['Latitude'].values

            if latitude_values.size > 0:
                latitude = latitude_values[0]
                # Prepare the data for CO2 emissions prediction
                input_data_co2 = np.array([[latitude, density_pred]])

                # Predict CO2 emissions
                co2_pred_log = co2_model.predict(input_data_co2)[0]
                co2_pred = math.exp(co2_pred_log)
               
            else:
                co2_pred = "Country not found or invalid data for CO2 prediction."
        else:
            population_pred = "Invalid country"
            density_pred = "Invalid country"
            co2_pred = "Invalid country"

    return render_template("home4.html", population_pred=population_pred, density_pred=density_pred, co2_pred=co2_pred)

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, port=5001)

 * Serving Flask app '__main__'
 * Debug mode: on


C:\Users\Dell\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:43:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\data\../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)
 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [01/Feb/2025 20:43:36] "GET / HTTP/1.1" 200 -
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Del